In [1]:
import os
import sys
from pyspark import SparkConf, SparkContext
# from pyspark.sql import SQLContext, Row
from pyspark.sql import *
from pyspark.sql.types import *


if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = '/opt/cloudera/parcels/CDH-5.7.1-1.cdh5.7.1.p0.11/lib/spark'
os.environ["PYSPARK_PYTHON"]="/home/leo/anaconda2/bin/python"
SPARK_HOME = os.environ['SPARK_HOME']
sys.path.insert(0, os.path.join(SPARK_HOME, "python"))

In [2]:
from pyspark import SparkContext,StorageLevel
sc = SparkContext( 'yarn-client', 'Try')
sqlContext = SQLContext(sc)

## broadcast variable

In [3]:
#broadcast var to others
text_node=[]
itemlist_node={}

# text_node=[u'\u3010']

# for ele in text_node:
#     ele.decode('utf-8')
#     print ele
text_node=['👍','●', '○','^^','◆','▲','✿','★','♥','■','◆','@@','☆','+','。','✔','＊','㊣','▼','◣','◢',
      '==============================','□','►','☉','^_^','‘×’','※','△','◇','♛','~^ ^~','~~~~~'
      ,'◎','*','Σ( ° △ °|||)︴'
     ,'〒','√','========== ==========','【','】','↖','↘','↗','↗','∥','^o^','ㄌ','ㄍ','ㄛ','ㄉ','╭','❤',
      '✈','￥','❒','✜''👍','●','➽','★','◆','■','▋','▌','❤','▲','♥','☻','㊣','❒','❺','❷','❹','❸','❶','〒','≧',
      '♞','➤','✦','✠','❉','✜','《','》','"',' ','🐻','☀','🎉','➕','🚫','📲','🌟','🌻','☆','★','◎']
text_node2=[]
for ele in text_node:
    ele=ele.decode('utf-8')
    text_node2.append(ele)
#     print ele,repr(ele)
# itemlist_node=[{'耳機':'earphone'},{'男香水':'m_perfume'},{'筆記型電腦':'notebook'},{'領帶':'necktie'},{'玩偶':'doll'},\
#          {'美容':'beauty'},{'自行車':'bike'},{'相機':'camera'},{'咖啡':'coffee'},{'茶':'tea'},{'錢包':'wallets'},\
#          {'果汁':'juice'},{'印章':'seal'},{'杯子':'cup'},{'茶具':'tea_set'},{'茶壺':'teapot'},{'護髮':'hair_care'},\
#          {'按摩':'massage'},{'麵包機':'breadmaker'},{'鋼筆':'fountain_pen'},{'豆漿':'soybean_milk'},\
#          {'抱枕':'pillow'},{'咖啡機':'coffee_machine'},{'智能手錶':'smartwatch'},{'果乾':'dried_fruit'},\
#          {'精油':'essential_oils'},{'剃毛':'shaving'},{'背包':'backpack'},{'咖啡杯':'mug'},{'烤爐':'oven'},\
#          {'女香水':'w_perfume'}]


text = sc.broadcast(text_node2)
itemlist= sc.broadcast(itemlist_node)

In [8]:
import re
import time
print time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) 

def cleanWord(line):
     
#     line.encode('utf-8')
    content=''
    pattern = re.sub(r'\{.*\}','',line)
    content=content+pattern
#     print content

    for element in text.value:
        if element in content:
            content=content.replace(element, "")

    content=content.replace('<![CDATA[/varcontext=', "")
    pattern_final = re.sub(r'NEW','0',content)
    pattern_final = re.sub(r'\'','0',pattern_final)
    return pattern_final


def createid(line):
    mylist_id=line.split('show?')[1]
    return mylist_id

def fixPrice(line):
    if int(line) >1000000:
        return 0
    else:
        return line

    
lines = sc.textFile("/user/admin/admin3/")
people = lines.map(lambda x: x.split("_|"))

# The schema is encoded in a string.
schemaString = "Title PRO_webads left_num sold_num Price SELLER_NAME SELLER_lastlogin SELLER_assess PRO_lastpricetime q_times price_times pic_web inner_content extra"

fields = [StructField(field_name,StringType(),True) for field_name in schemaString.split(" ")]
schema = StructType(fields)

# Apply the schema to the RDD.
schemaPeople = sqlContext.createDataFrame(people, schema)

# Register the DataFrame as a table.
schemaPeople.registerTempTable("people")

# SQL can be run over DataFrames that have been registered as a table.
results = sqlContext.sql("SELECT Title, PRO_webads, left_num, sold_num, Price, SELLER_NAME, SELLER_lastlogin,SELLER_assess, PRO_lastpricetime, q_times, price_times, pic_web, inner_content FROM people")

# The results of SQL queries are RDDs and support all the normal RDD operations.
names = results.map(lambda p: createid(p.PRO_webads)+ " "+ cleanWord(p.Title)+ " "+ p.PRO_webads+ " "+p.left_num+ " "+ p.sold_num + " "+fixPrice(p.Price)+ " "+ p.SELLER_NAME + " "+p.SELLER_lastlogin+ " "+ p.SELLER_assess+ " "+ p.PRO_lastpricetime + " "+p.q_times + " "+p.price_times + " "+p.pic_web + " "+cleanWord(p.inner_content))

catchdate=time.strftime("%d") # %m %Y
names.saveAsTextFile('/user/admin/output/{}'.format(catchdate))
# for name in names.collect():
#     print name

# # http://goods.ruten.com.tw/item/show?21533389987301
    
print time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) 



2016-07-26 03:09:05
21533389987301 JKATOP全新正品打結OUT拉鍊金屬耳機頭重低音麥克風線控耳塞式魔聲音感iphone6s/6plus http://goods.ruten.com.tw/item/show?21533389987301 801 198 99 comeonpeace 上次登入：2016-06-22 9435 2016-06-1919:29:14 27 165 http://f.rimg.com.tw/s1/f/19/e5/21533389987301_206_m.gif 露天拍賣-台灣NO.1拍賣網站
21551396166623 御美妍SarahJessicaParkerLovely慾望城市女性淡香100ML@另有迪奧嬌蘭 http://goods.ruten.com.tw/item/show?21551396166623 42 0 850 bestmask 上次登入：2016-06-21 73008  0 0 http://a.rimg.com.tw/s1/7/cf/df/21551396166623_614_m.jpg 露天拍賣-台灣NO.1拍賣網站品牌名稱SarahJessicaParkerLovely商品名稱慾望城市女性淡香容量與規格單瓶100ML其他特殊說明香調：柔美花香調 前味：佛手柑、花梨木、柑橘、薰衣草、蘋果馬丁尼中味：白水仙、廣藿香、蘭花 後味：木質香、香柏木、龍涎香、麝香下標前請先詢問有無現貨唷備註說明請置於陰涼乾燥處，避免陽光直接照射，使用時若產生不適症狀，請停止使用，並諮詢醫生意見消耗性產品如經拆封使用，恕無法退貨
2016-07-26 03:09:12
